In [2]:
import os
import numpy as np
import random
import matplotlib.pyplot as plt
import tensorflow as tf
from itertools import product
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, ZeroPadding2D, Activation, Dropout
from tensorflow_model_optimization.quantization.keras import vitis_quantize
import tensorflow_model_optimization as tfmot

shape_range =range(32,352,32)

for shape_x in shape_range:
    
    shape_y = shape_x
#     X_train=np.random.randint(0,255,size=[60,shape_x,shape_y,1])
#     Y_train=np.random.randint(0,10,size=[60,])
    X_test=np.random.randint(0,255,size=[10,shape_x,shape_y,1])
    Y_test=np.random.randint(0,10,size=[10,])
#     X_train = X_train.astype('float32')
    X_test = X_test.astype('float32')
#     X_train /= 255.0
    X_test /= 255.0


    inputs = Input(shape=(shape_x, shape_y, 1))
    for step in range(1,11):
        zeroPadding1 = ZeroPadding2D(padding=1)(inputs)
        conv1_kernel_num = random.randrange(32,64,2)
        conv1 = Conv2D(conv1_kernel_num, (3,3), padding='same', activation='relu')(zeroPadding1)
        pool1 = MaxPooling2D(pool_size=2)(conv1)

        zeroPadding2 = ZeroPadding2D(padding=1)(pool1)
        conv2_kernel_num = random.randrange(conv1_kernel_num,128,2)
        conv2 = Conv2D(conv2_kernel_num, (3,3), padding='same', activation='relu')(zeroPadding2)
        pool2 = MaxPooling2D(pool_size=2)(conv2)

        zeroPadding3 = ZeroPadding2D(padding=1)(pool2)
        conv3_kernel_num = random.randrange(conv2_kernel_num,256,2)
        conv3 = Conv2D(conv3_kernel_num, (3,3), padding='same', activation='relu')(zeroPadding3)

        zeroPadding4 = ZeroPadding2D(padding=1)(conv3)
        conv4_kernel_num = random.randrange(conv3_kernel_num,256,2)
        conv4 = Conv2D(conv4_kernel_num, (3,3), padding='same', activation='relu')(zeroPadding4)
        pool3 = MaxPooling2D(pool_size=2)(conv4)

        zeroPadding5 = ZeroPadding2D(padding=1)(pool3)
        conv5_kernel_num = random.randrange(conv4_kernel_num,512,2)
        conv5 = Conv2D(conv5_kernel_num, (3,3), padding='same', activation='relu')(zeroPadding5)

        zeroPadding6 = ZeroPadding2D(padding=1)(conv5)
        conv6_kernel_num = random.randrange(conv5_kernel_num,512,2)
        conv6 = Conv2D(conv6_kernel_num, (3,3), padding='same', activation='relu')(zeroPadding6)
        pool4 = MaxPooling2D(pool_size=2)(conv6)

        zeroPadding7 = ZeroPadding2D(padding=1)(pool4)
        conv7_kernel_num = random.randrange(conv5_kernel_num,512,2)
        conv7 = Conv2D(conv7_kernel_num, (3,3), padding='same', activation='relu')(zeroPadding7)

        zeroPadding8 = ZeroPadding2D(padding=1)(conv7)
        conv8_kernel_num = random.randrange(conv7_kernel_num,512,2)
        conv8 = Conv2D(conv8_kernel_num, (3,3), padding='same', activation='relu')(zeroPadding8)
        pool5 = MaxPooling2D(pool_size=2)(conv8)

        conv9_kernel_num = random.randrange(conv8_kernel_num,3072,2)
        conv9 = Conv2D(conv9_kernel_num, (7,7), padding='same', activation='relu')(pool5)
        drop1 = Dropout(0.5)(conv9)

        conv10_kernel_num = random.randrange(conv9_kernel_num,3072,2)
        conv10 = Conv2D(conv10_kernel_num, (1,1), padding='same', activation='relu')(pool5)
        drop2 = Dropout(0.5)(conv10)

        conv11 = Conv2D(10, (1,1))(drop2)

        flat = Flatten()(conv11)

        outputs = Dense(10, activation = 'softmax')(flat)


        model = Model(inputs=inputs, outputs=outputs)
        model.summary()
        model.save('./vgg11_h5/vgg11_{input_x}_{input_y}_{step}.h5'.format(input_x=shape_x,input_y=shape_y,step=step))
#         model.compile(optimizer='adam',
#                   loss="sparse_categorical_crossentropy", 
#                   metrics=['accuracy'])
#         history = model.fit(X_train, Y_train, epochs=1)
#         model.get_weights()[0].dtype
        quantizer = vitis_quantize.VitisQuantizer(model)
        quantized_model = quantizer.quantize_model(calib_dataset = X_test[1:8], weight_bit=8, activation_bit=8)
#         quantized_model.compile(loss='sparse_categorical_crossentropy', metrics=["accuracy"])
        quantized_model.save('quantized.h5')
        !vai_c_tensorflow2 \
                --model ./quantized.h5 \
                --arch ./arch.json \
                --output_dir ./xmodel_vgg11 \
                --net_name deploy
        os.rename("./xmodel_vgg11/deploy.xmodel","./xmodel_vgg11/vgg11_{input_x}_{input_y}_{step}.xmodel".format(input_x=shape_x,input_y=shape_y,step=step))

Model: "model_100"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_11 (InputLayer)        [(None, 32, 32, 1)]       0         
_________________________________________________________________
zero_padding2d_800 (ZeroPadd (None, 34, 34, 1)         0         
_________________________________________________________________
conv2d_1100 (Conv2D)         (None, 34, 34, 42)        420       
_________________________________________________________________
max_pooling2d_500 (MaxPoolin (None, 17, 17, 42)        0         
_________________________________________________________________
zero_padding2d_801 (ZeroPadd (None, 19, 19, 42)        0         
_________________________________________________________________
conv2d_1101 (Conv2D)         (None, 19, 19, 54)        20466     
_________________________________________________________________
max_pooling2d_501 (MaxPoolin (None, 9, 9, 54)          0 

[VAI INFO] Update custom_layer_type: []
[VAI INFO] Update activation_bit: 8
[VAI INFO] Update weight_bit: 8
[VAI INFO] Start CrossLayerEqualization...
10/10 [==============================] - 3s 322ms/step
[VAI INFO] CrossLayerEqualization Done.
[VAI INFO] Start Quantize Calibration...
1/1 [==============================] - 5s 5s/step
[VAI INFO] Quantize Calibration Done.
[VAI INFO] Start Post-Quantize Adjustment...
[VAI INFO] Post-Quantize Adjustment Done.
[VAI INFO] Quantization Finished.
**************************************************
* VITIS_AI Compilation - Xilinx Inc.
**************************************************
[INFO] Namespace(batchsize=1, inputs_shape=None, layout='NHWC', model_files=['./quantized.h5'], model_type='tensorflow2', named_inputs_shape=None, out_filename='/tmp/deploy_org.xmodel', proto=None)
[INFO] tensorflow2 model: /workspace/DPU-PYNQ/host/quantized.h5
[INFO] keras version: 2.6.0
[INFO] Tensorflow Keras model type: functional
[INFO] generate xmodel     :

Model: "model_103"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_11 (InputLayer)        [(None, 32, 32, 1)]       0         
_________________________________________________________________
zero_padding2d_824 (ZeroPadd (None, 34, 34, 1)         0         
_________________________________________________________________
conv2d_1133 (Conv2D)         (None, 34, 34, 58)        580       
_________________________________________________________________
max_pooling2d_515 (MaxPoolin (None, 17, 17, 58)        0         
_________________________________________________________________
zero_padding2d_825 (ZeroPadd (None, 19, 19, 58)        0         
_________________________________________________________________
conv2d_1134 (Conv2D)         (None, 19, 19, 72)        37656     
_________________________________________________________________
max_pooling2d_516 (MaxPoolin (None, 9, 9, 72)          0 

[VAI INFO] Update custom_layer_type: []
[VAI INFO] Update activation_bit: 8
[VAI INFO] Update weight_bit: 8
[VAI INFO] Start CrossLayerEqualization...
10/10 [==============================] - 2s 201ms/step
[VAI INFO] CrossLayerEqualization Done.
[VAI INFO] Start Quantize Calibration...
1/1 [==============================] - 4s 4s/step
[VAI INFO] Quantize Calibration Done.
[VAI INFO] Start Post-Quantize Adjustment...
[VAI INFO] Post-Quantize Adjustment Done.
[VAI INFO] Quantization Finished.
**************************************************
* VITIS_AI Compilation - Xilinx Inc.
**************************************************
[INFO] Namespace(batchsize=1, inputs_shape=None, layout='NHWC', model_files=['./quantized.h5'], model_type='tensorflow2', named_inputs_shape=None, out_filename='/tmp/deploy_org.xmodel', proto=None)
[INFO] tensorflow2 model: /workspace/DPU-PYNQ/host/quantized.h5
[INFO] keras version: 2.6.0
[INFO] Tensorflow Keras model type: functional
[INFO] generate xmodel     :

[UNILOG][INFO] The compiled xmodel is saved to "/workspace/DPU-PYNQ/host/./xmodel_vgg11/deploy.xmodel"
[UNILOG][INFO] The compiled xmodel's md5sum is b8ac493f01be869f16976711d3d26dbb, and has been saved to "/workspace/DPU-PYNQ/host/./xmodel_vgg11/md5sum.txt"
Model: "model_106"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_11 (InputLayer)        [(None, 32, 32, 1)]       0         
_________________________________________________________________
zero_padding2d_848 (ZeroPadd (None, 34, 34, 1)         0         
_________________________________________________________________
conv2d_1166 (Conv2D)         (None, 34, 34, 54)        540       
_________________________________________________________________
max_pooling2d_530 (MaxPoolin (None, 17, 17, 54)        0         
_________________________________________________________________
zero_padding2d_849 (ZeroPadd (None, 19, 19, 54)        0      

...

Model: "model_193"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_20 (InputLayer)        [(None, 320, 320, 1)]     0         
_________________________________________________________________
zero_padding2d_1544 (ZeroPad (None, 322, 322, 1)       0         
_________________________________________________________________
conv2d_2123 (Conv2D)         (None, 322, 322, 40)      400       
_________________________________________________________________
max_pooling2d_965 (MaxPoolin (None, 161, 161, 40)      0         
_________________________________________________________________
zero_padding2d_1545 (ZeroPad (None, 163, 163, 40)      0         
_________________________________________________________________
conv2d_2124 (Conv2D)         (None, 163, 163, 78)      28158     
_________________________________________________________________
max_pooling2d_966 (MaxPoolin (None, 81, 81, 78)        0 

[VAI INFO] Update custom_layer_type: []
[VAI INFO] Update activation_bit: 8
[VAI INFO] Update weight_bit: 8
[VAI INFO] Start CrossLayerEqualization...
10/10 [==============================] - 2s 203ms/step
[VAI INFO] CrossLayerEqualization Done.
[VAI INFO] Start Quantize Calibration...
1/1 [==============================] - 7s 7s/step
[VAI INFO] Quantize Calibration Done.
[VAI INFO] Start Post-Quantize Adjustment...
[VAI INFO] Post-Quantize Adjustment Done.
[VAI INFO] Quantization Finished.
**************************************************
* VITIS_AI Compilation - Xilinx Inc.
**************************************************
[INFO] Namespace(batchsize=1, inputs_shape=None, layout='NHWC', model_files=['./quantized.h5'], model_type='tensorflow2', named_inputs_shape=None, out_filename='/tmp/deploy_org.xmodel', proto=None)
[INFO] tensorflow2 model: /workspace/DPU-PYNQ/host/quantized.h5
[INFO] keras version: 2.6.0
[INFO] Tensorflow Keras model type: functional
[INFO] generate xmodel     :

Model: "model_196"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_20 (InputLayer)        [(None, 320, 320, 1)]     0         
_________________________________________________________________
zero_padding2d_1568 (ZeroPad (None, 322, 322, 1)       0         
_________________________________________________________________
conv2d_2156 (Conv2D)         (None, 322, 322, 58)      580       
_________________________________________________________________
max_pooling2d_980 (MaxPoolin (None, 161, 161, 58)      0         
_________________________________________________________________
zero_padding2d_1569 (ZeroPad (None, 163, 163, 58)      0         
_________________________________________________________________
conv2d_2157 (Conv2D)         (None, 163, 163, 76)      39748     
_________________________________________________________________
max_pooling2d_981 (MaxPoolin (None, 81, 81, 76)        0 

[VAI INFO] Update custom_layer_type: []
[VAI INFO] Update activation_bit: 8
[VAI INFO] Update weight_bit: 8
[VAI INFO] Start CrossLayerEqualization...
10/10 [==============================] - 2s 201ms/step
[VAI INFO] CrossLayerEqualization Done.
[VAI INFO] Start Quantize Calibration...
1/1 [==============================] - 6s 6s/step
[VAI INFO] Quantize Calibration Done.
[VAI INFO] Start Post-Quantize Adjustment...
[VAI INFO] Post-Quantize Adjustment Done.
[VAI INFO] Quantization Finished.
**************************************************
* VITIS_AI Compilation - Xilinx Inc.
**************************************************
[INFO] Namespace(batchsize=1, inputs_shape=None, layout='NHWC', model_files=['./quantized.h5'], model_type='tensorflow2', named_inputs_shape=None, out_filename='/tmp/deploy_org.xmodel', proto=None)
[INFO] tensorflow2 model: /workspace/DPU-PYNQ/host/quantized.h5
[INFO] keras version: 2.6.0
[INFO] Tensorflow Keras model type: functional
[INFO] generate xmodel     :

Model: "model_199"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_20 (InputLayer)        [(None, 320, 320, 1)]     0         
_________________________________________________________________
zero_padding2d_1592 (ZeroPad (None, 322, 322, 1)       0         
_________________________________________________________________
conv2d_2189 (Conv2D)         (None, 322, 322, 52)      520       
_________________________________________________________________
max_pooling2d_995 (MaxPoolin (None, 161, 161, 52)      0         
_________________________________________________________________
zero_padding2d_1593 (ZeroPad (None, 163, 163, 52)      0         
_________________________________________________________________
conv2d_2190 (Conv2D)         (None, 163, 163, 54)      25326     
_________________________________________________________________
max_pooling2d_996 (MaxPoolin (None, 81, 81, 54)        0 

In [3]:
import os
import numpy as np
import random
import matplotlib.pyplot as plt
import tensorflow as tf
from itertools import product
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, ZeroPadding2D, Activation, Dropout
from tensorflow_model_optimization.quantization.keras import vitis_quantize
import tensorflow_model_optimization as tfmot

shape_range =range(48,220,34)

for shape_x in shape_range:
    
    shape_y = shape_x
#     X_train=np.random.randint(0,255,size=[60,shape_x,shape_y,1])
#     Y_train=np.random.randint(0,10,size=[60,])
    X_test=np.random.randint(0,255,size=[60,shape_x,shape_y,1])
    Y_test=np.random.randint(0,10,size=[60,])
#     X_train = X_train.astype('float32')
    X_test = X_test.astype('float32')
#     X_train /= 255.0
    X_test /= 255.0

    max_step =4
    inputs = Input(shape=(shape_x, shape_y, 1))
    for step in range(1,max_step+1):
        zeroPadding1 = ZeroPadding2D(padding=1)(inputs)
        conv1_kernel_num = int(32+step*32/max_step)
        conv1 = Conv2D(conv1_kernel_num, (3,3), padding='same', activation='relu')(zeroPadding1)
        pool1 = MaxPooling2D(pool_size=2)(conv1)

        zeroPadding2 = ZeroPadding2D(padding=1)(pool1)
        conv2_kernel_num = int(64+step*64/max_step)
        conv2 = Conv2D(conv2_kernel_num, (3,3), padding='same', activation='relu')(zeroPadding2)
        pool2 = MaxPooling2D(pool_size=2)(conv2)

        zeroPadding3 = ZeroPadding2D(padding=1)(pool2)
        conv3_kernel_num = int(128+step*128/max_step)
        conv3 = Conv2D(conv3_kernel_num, (3,3), padding='same', activation='relu')(zeroPadding3)

        zeroPadding4 = ZeroPadding2D(padding=1)(conv3)
        conv4_kernel_num = int(128+step*128/max_step)
        conv4 = Conv2D(conv4_kernel_num, (3,3), padding='same', activation='relu')(zeroPadding4)
        pool3 = MaxPooling2D(pool_size=2)(conv4)

        zeroPadding5 = ZeroPadding2D(padding=1)(pool3)
        conv5_kernel_num = int(256+step*256/max_step)
        conv5 = Conv2D(conv5_kernel_num, (3,3), padding='same', activation='relu')(zeroPadding5)

        zeroPadding6 = ZeroPadding2D(padding=1)(conv5)
        conv6_kernel_num = int(256+step*256/max_step)
        conv6 = Conv2D(conv6_kernel_num, (3,3), padding='same', activation='relu')(zeroPadding6)
        pool4 = MaxPooling2D(pool_size=2)(conv6)

        zeroPadding7 = ZeroPadding2D(padding=1)(pool4)
        conv7_kernel_num = int(256+step*256/max_step)
        conv7 = Conv2D(conv7_kernel_num, (3,3), padding='same', activation='relu')(zeroPadding7)

        zeroPadding8 = ZeroPadding2D(padding=1)(conv7)
        conv8_kernel_num = int(256+step*256/max_step)
        conv8 = Conv2D(conv8_kernel_num, (3,3), padding='same', activation='relu')(zeroPadding8)
        pool5 = MaxPooling2D(pool_size=2)(conv8)

        conv9_kernel_num = int(512+step*2560/max_step)
        conv9 = Conv2D(conv9_kernel_num, (7,7), padding='same', activation='relu')(pool5)
        drop1 = Dropout(0.5)(conv9)

        conv10_kernel_num = int(512+step*2560/max_step)
        conv10 = Conv2D(conv10_kernel_num, (1,1), padding='same', activation='relu')(pool5)
        drop2 = Dropout(0.5)(conv10)

        conv11 = Conv2D(10, (1,1))(drop2)

        flat = Flatten()(conv11)
        
        outputs = Dense(10, activation = 'softmax')(flat)

        


        model = Model(inputs=inputs, outputs=outputs)
        model.summary()
        model.save('./vgg11_h5/vgg11_{input_x}_{input_y}_{step}.h5'.format(input_x=shape_x,input_y=shape_y,step=step))
#         model.compile(optimizer='adam',
#                   loss="sparse_categorical_crossentropy", 
#                   metrics=['accuracy'])
#         history = model.fit(X_train, Y_train, epochs=1)
#         model.get_weights()[0].dtype
        quantizer = vitis_quantize.VitisQuantizer(model)
        quantized_model = quantizer.quantize_model(calib_dataset = X_test[1:8], weight_bit=8, activation_bit=8)
#         quantized_model.compile(loss='sparse_categorical_crossentropy', metrics=["accuracy"])
        quantized_model.save('quantized.h5')
        !vai_c_tensorflow2 \
                --model ./quantized.h5 \
                --arch ./arch.json \
                --output_dir ./xmodel_vgg11 \
                --net_name deploy
        os.rename("./xmodel_vgg11/deploy.xmodel","./xmodel_vgg11/vgg11_{input_x}_{input_y}_{step}.xmodel".format(input_x=shape_x,input_y=shape_y,step=step))

Model: "model_28"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         [(None, 48, 48, 1)]       0         
_________________________________________________________________
zero_padding2d_224 (ZeroPadd (None, 50, 50, 1)         0         
_________________________________________________________________
conv2d_308 (Conv2D)          (None, 50, 50, 40)        400       
_________________________________________________________________
max_pooling2d_140 (MaxPoolin (None, 25, 25, 40)        0         
_________________________________________________________________
zero_padding2d_225 (ZeroPadd (None, 27, 27, 40)        0         
_________________________________________________________________
conv2d_309 (Conv2D)          (None, 27, 27, 80)        28880     
_________________________________________________________________
max_pooling2d_141 (MaxPoolin (None, 13, 13, 80)        0  

[VAI INFO] Update custom_layer_type: []
[VAI INFO] Update activation_bit: 8
[VAI INFO] Update weight_bit: 8
[VAI INFO] Start CrossLayerEqualization...
10/10 [==============================] - 2s 237ms/step
[VAI INFO] CrossLayerEqualization Done.
[VAI INFO] Start Quantize Calibration...
1/1 [==============================] - 4s 4s/step
[VAI INFO] Quantize Calibration Done.
[VAI INFO] Start Post-Quantize Adjustment...
[VAI INFO] Post-Quantize Adjustment Done.
[VAI INFO] Quantization Finished.
**************************************************
* VITIS_AI Compilation - Xilinx Inc.
**************************************************
[INFO] Namespace(batchsize=1, inputs_shape=None, layout='NHWC', model_files=['./quantized.h5'], model_type='tensorflow2', named_inputs_shape=None, out_filename='/tmp/deploy_org.xmodel', proto=None)
[INFO] tensorflow2 model: /workspace/DPU-PYNQ/host/quantized.h5
[INFO] keras version: 2.6.0
[INFO] Tensorflow Keras model type: functional
[INFO] generate xmodel     :

Model: "model_31"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         [(None, 48, 48, 1)]       0         
_________________________________________________________________
zero_padding2d_248 (ZeroPadd (None, 50, 50, 1)         0         
_________________________________________________________________
conv2d_341 (Conv2D)          (None, 50, 50, 64)        640       
_________________________________________________________________
max_pooling2d_155 (MaxPoolin (None, 25, 25, 64)        0         
_________________________________________________________________
zero_padding2d_249 (ZeroPadd (None, 27, 27, 64)        0         
_________________________________________________________________
conv2d_342 (Conv2D)          (None, 27, 27, 128)       73856     
_________________________________________________________________
max_pooling2d_156 (MaxPoolin (None, 13, 13, 128)       0  

[VAI INFO] Update custom_layer_type: []
[VAI INFO] Update activation_bit: 8
[VAI INFO] Update weight_bit: 8
[VAI INFO] Start CrossLayerEqualization...
10/10 [==============================] - 2s 233ms/step
[VAI INFO] CrossLayerEqualization Done.
[VAI INFO] Start Quantize Calibration...
1/1 [==============================] - 4s 4s/step
[VAI INFO] Quantize Calibration Done.
[VAI INFO] Start Post-Quantize Adjustment...
[VAI INFO] Post-Quantize Adjustment Done.
[VAI INFO] Quantization Finished.
**************************************************
* VITIS_AI Compilation - Xilinx Inc.
**************************************************
[INFO] Namespace(batchsize=1, inputs_shape=None, layout='NHWC', model_files=['./quantized.h5'], model_type='tensorflow2', named_inputs_shape=None, out_filename='/tmp/deploy_org.xmodel', proto=None)
[INFO] tensorflow2 model: /workspace/DPU-PYNQ/host/quantized.h5
[INFO] keras version: 2.6.0
[INFO] Tensorflow Keras model type: functional
[INFO] generate xmodel     :

[UNILOG][INFO] The compiled xmodel's md5sum is b7c78135d89c867aab2005d8cbefe156, and has been saved to "/workspace/DPU-PYNQ/host/./xmodel_vgg11/md5sum.txt"
Model: "model_34"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         [(None, 82, 82, 1)]       0         
_________________________________________________________________
zero_padding2d_272 (ZeroPadd (None, 84, 84, 1)         0         
_________________________________________________________________
conv2d_374 (Conv2D)          (None, 84, 84, 56)        560       
_________________________________________________________________
max_pooling2d_170 (MaxPoolin (None, 42, 42, 56)        0         
_________________________________________________________________
zero_padding2d_273 (ZeroPadd (None, 44, 44, 56)        0         
_________________________________________________________________
conv2d_375 (Conv2D)          (None

...

[UNILOG][INFO] The compiled xmodel's md5sum is f3214695ee7b053af951259790fb8326, and has been saved to "/workspace/DPU-PYNQ/host/./xmodel_vgg11/md5sum.txt"
Model: "model_46"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_12 (InputLayer)        [(None, 184, 184, 1)]     0         
_________________________________________________________________
zero_padding2d_368 (ZeroPadd (None, 186, 186, 1)       0         
_________________________________________________________________
conv2d_506 (Conv2D)          (None, 186, 186, 56)      560       
_________________________________________________________________
max_pooling2d_230 (MaxPoolin (None, 93, 93, 56)        0         
_________________________________________________________________
zero_padding2d_369 (ZeroPadd (None, 95, 95, 56)        0         
_________________________________________________________________
conv2d_507 (Conv2D)          (None

[VAI INFO] Update custom_layer_type: []
[VAI INFO] Update activation_bit: 8
[VAI INFO] Update weight_bit: 8
[VAI INFO] Start CrossLayerEqualization...
10/10 [==============================] - 2s 205ms/step
[VAI INFO] CrossLayerEqualization Done.
[VAI INFO] Start Quantize Calibration...
1/1 [==============================] - 5s 5s/step
[VAI INFO] Quantize Calibration Done.
[VAI INFO] Start Post-Quantize Adjustment...
[VAI INFO] Post-Quantize Adjustment Done.
[VAI INFO] Quantization Finished.
**************************************************
* VITIS_AI Compilation - Xilinx Inc.
**************************************************
[INFO] Namespace(batchsize=1, inputs_shape=None, layout='NHWC', model_files=['./quantized.h5'], model_type='tensorflow2', named_inputs_shape=None, out_filename='/tmp/deploy_org.xmodel', proto=None)
[INFO] tensorflow2 model: /workspace/DPU-PYNQ/host/quantized.h5
[INFO] keras version: 2.6.0
[INFO] Tensorflow Keras model type: functional
[INFO] generate xmodel     :

Model: "model_49"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_13 (InputLayer)        [(None, 218, 218, 1)]     0         
_________________________________________________________________
zero_padding2d_392 (ZeroPadd (None, 220, 220, 1)       0         
_________________________________________________________________
conv2d_539 (Conv2D)          (None, 220, 220, 48)      480       
_________________________________________________________________
max_pooling2d_245 (MaxPoolin (None, 110, 110, 48)      0         
_________________________________________________________________
zero_padding2d_393 (ZeroPadd (None, 112, 112, 48)      0         
_________________________________________________________________
conv2d_540 (Conv2D)          (None, 112, 112, 96)      41568     
_________________________________________________________________
max_pooling2d_246 (MaxPoolin (None, 56, 56, 96)        0  

[VAI INFO] Update custom_layer_type: []
[VAI INFO] Update activation_bit: 8
[VAI INFO] Update weight_bit: 8
[VAI INFO] Start CrossLayerEqualization...
10/10 [==============================] - 2s 200ms/step
[VAI INFO] CrossLayerEqualization Done.
[VAI INFO] Start Quantize Calibration...
1/1 [==============================] - 5s 5s/step
[VAI INFO] Quantize Calibration Done.
[VAI INFO] Start Post-Quantize Adjustment...
[VAI INFO] Post-Quantize Adjustment Done.
[VAI INFO] Quantization Finished.
**************************************************
* VITIS_AI Compilation - Xilinx Inc.
**************************************************
[INFO] Namespace(batchsize=1, inputs_shape=None, layout='NHWC', model_files=['./quantized.h5'], model_type='tensorflow2', named_inputs_shape=None, out_filename='/tmp/deploy_org.xmodel', proto=None)
[INFO] tensorflow2 model: /workspace/DPU-PYNQ/host/quantized.h5
[INFO] keras version: 2.6.0
[INFO] Tensorflow Keras model type: functional
[INFO] generate xmodel     :

[UNILOG][INFO] The compiled xmodel is saved to "/workspace/DPU-PYNQ/host/./xmodel_vgg11/deploy.xmodel"
[UNILOG][INFO] The compiled xmodel's md5sum is 3db5648388b3fa3f4aa7214fe4458b21, and has been saved to "/workspace/DPU-PYNQ/host/./xmodel_vgg11/md5sum.txt"
